<a href="https://colab.research.google.com/github/divyap2706/Big-Data-Project-X/blob/main/BigData_BridgesTunnels_DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+git://github.com/VIDA-NYU/openclean.git
from openclean.pipeline import stream


  Cloning git://github.com/VIDA-NYU/openclean.git to /tmp/pip-req-build-yixfukui
  Running command git clone -q git://github.com/VIDA-NYU/openclean.git /tmp/pip-req-build-yixfukui
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 266kB 4.3MB/s 
     |████████████████████████████████| 102kB 6.7MB/s 
     |████████████████████████████████| 235kB 15.0MB/s 
     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 92kB 6.0MB/s 
     |████████████████████████████████| 532kB 15.1MB/s 
     |████████████████████████████████| 163kB 20.3MB/s 
     |████████████████████████████████| 153kB 21.1MB/s 
     |████████████████████████████████| 71kB 5.9MB/s 
  Created wheel for openclean: filename=openclean-0.1.0-cp37-none-any.whl size=4897 sha256=e80b41bdd22acdafeddc978489b06eae0dfd6767e2206bffeee5b8ad352b5153
  Stored in directory: /tmp/pip-ephem-wheel-cache-y56zuuq8/wheels/5f/de/0d/3f94f0323cd128a09f597f9c646d4d57

In [3]:
# Import the data and then use OpenClean's stream feature so that we do not need
# to put the entire file into memory
datafile = "/content/drive/MyDrive/Big Data/Big Data Project/Data Max is Working on/Hourly_Traffic_on_Metropolitan_Transportation_Authority__MTA__Bridges_and_Tunnels__Beginning_2010.csv"
ds = stream(datafile)

# Print the first ten rows of the dataset to get a first
# idea of the content.
ds.head()

,Plaza ID,Date,Hour,Direction,# Vehicles - E-ZPass,# Vehicles - VToll
0,3,02/08/2017,6,O,3524,446
1,1,02/08/2017,17,O,2019,400
2,11,02/08/2017,15,I,5999,498
3,1,02/06/2017,3,O,293,94
4,1,02/06/2017,14,O,1881,391
5,9,02/09/2017,10,O,309,44
6,2,02/11/2017,5,I,455,85
7,1,02/09/2017,8,O,541,106
8,9,02/06/2017,22,O,1120,242
9,5,02/07/2017,18,O,358,48


In [4]:
# Profile rows using the default data profiler.

from openclean.profiling.column import DefaultColumnProfiler

profiles = ds.profile(default_profiler=DefaultColumnProfiler)

In [5]:
# Print overview of profiling results.

profiles.stats()

,total,empty,distinct,uniqueness,entropy
Plaza ID,1728845,0,20,0.000012,4.164765
Date,1728845,0,4024,0.002328,11.955525
Hour,1728845,0,24,0.000014,4.584962
Direction,1728845,0,2,0.000001,0.999681
# Vehicles - E-ZPass,1728845,0,8997,0.005204,11.804486
# Vehicles - VToll,1728845,0,1770,0.001024,9.486544


In [6]:
#Print the distinct values of a column
print(ds.distinct('Plaza ID'))
print(ds.distinct('Hour'))
print(ds.distinct('Direction'))

Counter({'3': 135552, '9': 135552, '1': 130896, '2': 130896, '5': 128688, '6': 128688, '7': 123072, '8': 122784, '4': 120624, '11': 66000, '28': 59221, '27': 59221, '24': 59202, '26': 59176, '25': 58941, '29': 58210, '23': 58092, '30': 32249, '21': 32207, '22': 29574})
Counter({'15': 72055, '14': 72055, '8': 72055, '13': 72055, '12': 72055, '11': 72055, '6': 72054, '10': 72054, '7': 72054, '17': 72053, '5': 72053, '16': 72053, '9': 72053, '4': 72052, '18': 72050, '19': 72047, '20': 72046, '22': 72045, '21': 72044, '23': 72042, '0': 71999, '3': 71976, '1': 71954, '2': 71886})
Counter({'I': 882586, 'O': 846259})


In [7]:
# look at all the datatypes in the dataset
print(profiles.types())

                      date   int  str
Plaza ID                 0    20    0
Date                  4024     0    0
Hour                     0    24    0
Direction                0     0    2
# Vehicles - E-ZPass     0  8997    0
# Vehicles - VToll       0  1770    0


In [8]:
# query the minimum and maximum in the columns column
print(profiles.minmax('Plaza ID'))
print(profiles.minmax('Date'))
print(profiles.minmax('Hour'))
print(profiles.minmax('# Vehicles - E-ZPass'))
print(profiles.minmax('# Vehicles - VToll'))

     min  max
int    1   30
            min        max
date 2010-01-01 2021-03-20
     min  max
int    0   23
     min    max
int    0  16854
     min   max
int    0  2567


In [9]:
# Print the number of rows in the full dataset
print('\n{:,} rows.'.format(ds.count()))


1,728,845 rows.


In [16]:
# Filter out plaza ID # 22 and 11 because they only have data for one direction
from openclean.operator.transform.filter import filter
from openclean.function.eval.base import Col
# Verrazano-Narrows Bridge (VNB)
filterOut_Verazano = ds.filter(predicate=Col('Plaza ID')!='11')
# Filter out Robert F. Kennedy Bridge Manhattan Plaza (TBM)
filterOut = filterOut_Verazano.filter(predicate=Col('Plaza ID')!='22')
filterOut.head()

,Plaza ID,Date,Hour,Direction,# Vehicles - E-ZPass,# Vehicles - VToll
0,3,02/08/2017,6,O,3524,446
1,1,02/08/2017,17,O,2019,400
3,1,02/06/2017,3,O,293,94
4,1,02/06/2017,14,O,1881,391
5,9,02/09/2017,10,O,309,44
6,2,02/11/2017,5,I,455,85
7,1,02/09/2017,8,O,541,106
8,9,02/06/2017,22,O,1120,242
9,5,02/07/2017,18,O,358,48
11,1,02/10/2017,14,O,1699,416


There are only 10 actual plazas, and the Plaza IDs changed in 2017 from 1-11, to 21-30. We need to map the correpsonding IDs from each time period. 

In [18]:
# Define a function to fix plaza IDs
def plaza_transform(plaza):
  if plaza == '21':
    return '1'
  elif plaza == '22':
    return '2'
  elif plaza == '23':
    return '3'
  elif plaza == '24':
    return '4'
  elif plaza == '25':
    return '5'
  elif plaza == '26':
    return '6'
  elif plaza == '27':
    return '7'
  #The Hugh L. Carey Tunnel is the official name for the Brooklyn Battery Tunnel
  elif plaza == '28':
    return '8'
  elif plaza == '29':
    return '9'
  elif plaza == '30':
    return '11'
  else:
    return plaza

fixedPlazas = filterOut.update('Plaza ID', plaza_transform)
print(fixedPlazas.distinct('Plaza ID'))

Counter({'9': 193762, '3': 193644, '6': 187864, '5': 187629, '7': 182293, '8': 182005, '4': 179826, '1': 163103, '2': 130896, '11': 32249})


In [19]:
# Export data to file
# Note: Change the file path as needed
import pandas as pd

output = fixedPlazas.to_df()
output.to_csv(r'/content/drive/MyDrive/Big Data/Big Data Project/Data Max is Working on/New_Bridges_Tunnels.csv', index = False, header=True)

